Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra
import escher

# Load model

In [ ]:
model = cobra.io.load_json_model("./models/iMM904.json")

# Utilities

In [ ]:
def print_formulas(reaction):
    """Print formulas of reactants and products of a reaction."""
    print('reactants')
    for reactant in reaction.reactants:
        print(f'{reactant.id} ({reactant.name}): F {reactant.formula}')
    print('products')
    for product in reaction.products:
        print(f'{product.id} ({product.name}): F {product.formula}')

def print_formula_weights(reaction):
    """Print formula weights of reactants and products of a reaction."""
    print('reactants')
    for reactant in reaction.reactants:
        print(f'{reactant.id} ({reactant.name}): MW {reactant.formula_weight}')
    print('products')
    for product in reaction.products:
        print(f'{product.id} ({product.name}): MW {product.formula_weight}')
        
def print_stoichiometry(reaction):
    """Pretty-print stoichiometry of reaction."""
    for metabolite, coeff in reaction.metabolites.items():
        print(f'{coeff}, {metabolite.name}')

# Objective function

Get objective function (biomass/growth)

In [ ]:
biomass = model.reactions.BIOMASS_SC5_notrace
biomass

In [ ]:
print_stoichiometry(biomass)

> These stoichiometric constants were determined by the relative compositions (i.e. mass fractions) of these compounds in exponentially growing yeast, as determined from experiments (Mo et al., 2009).

Medium

In [ ]:
model.medium

Unrestrict bounds

In [ ]:
model.reactions.get_by_id('EX_glc__D_e').bounds = (-10, 0)
model.reactions.get_by_id('EX_o2_e').bounds = (-999999.0, 0)

> Problem: If I set the bounds of glucose exchange to be (-99999, 0), the objective function reaches a huge value, so I left it at the default value.

Linear reaction coefficients

In [ ]:
cobra.util.solver.linear_reaction_coefficients(model)

> The objective function is the biomass reaction and the biomass reaction only, not a linear combination of anything.

Optimise using (vanilla) FBA

In [ ]:
solution = model.optimize()

In [ ]:
model.summary()

In [ ]:
solution

# Fluxes

Import list of metabolites represented in biomass reaction.  `biomass_type` is manually labelled.

In [ ]:
biomass_metabolites_df = pd.read_csv('iMM904-biomass-categories.csv', delimiter=',')

In [ ]:
biomass_metabolites_df

Get fluxes of each metabolite in the (optimised) biomass reaction, append to the dataframe.

In [ ]:
biomass_metabolites_df['flux'] = [np.nan] * len(biomass_metabolites_df)

In [ ]:
for df_idx, metabolite_id in enumerate(biomass_metabolites_df.id):
    metabolite = model.metabolites.get_by_id(metabolite_id)
    # Inspect reactants, ignore products, based on stoichiometric constants
    if biomass.metabolites[metabolite] < 0:
        flux_in_biomass = metabolite.summary().consuming_flux.loc['BIOMASS_SC5_notrace'].flux
        #print(f'{metabolite.name}: {-1/flux_in_biomass}')
        biomass_metabolites_df['flux'].iloc[df_idx] = flux_in_biomass

In [ ]:
biomass_metabolites_df

Combined fluxes for each category of metabolite

In [ ]:
combined_fluxes = pd.pivot_table(
    biomass_metabolites_df,
    values='flux',
    index='biomass_type',
    aggfunc='sum',
    dropna=False,
)
combined_fluxes['reciprocal_flux'] = -1/combined_fluxes['flux']
combined_fluxes
# Ignore ngam